In [1]:
import string
import numpy as np
import pandas as pd
from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr

In [3]:
def post_processing(pred):

    ## only use the last 3 sentences
    #proc_pred=[]
    #for p in pred:
    #    splits=[s for s in p.lower().split('\n') if s != '']
    #    proc_pred.append(' '.join(splits[-3:]).translate(str.maketrans('', '', string.punctuation)))
    
    proc_pred=[]
    for p in pred:
        splits=[s for s in p.lower().split('\n') if s != '']
        proc_pred.append(splits[-1].translate(str.maketrans('', '', string.punctuation)))
    
    new_pred=[]
    for p in proc_pred:       
        if 'response' in p:
            try: new_pred.append([t for t in p.lower().split('response')[1].split() if t.isnumeric()][0])
            except: new_pred.append(2)
        elif 'output' in p:
            try: new_pred.append([t for t in p.lower().split('output')[1].split() if t.isnumeric()][0])
            except: new_pred.append(2)
        elif 'return' in p:
            try: new_pred.append([t for t in p.lower().split('return')[1].split() if t.isnumeric()][0])
            except: new_pred.append(2)
        elif 'result' in p:
            try: new_pred.append([t for t in p.lower().split('result')[1].split() if t.isnumeric()][0])
            except: new_pred.append(2)                
        else:
            try: new_pred.append(p.lower().split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [4]:
ds = load_dataset("beanham/spatial_union_dataset")
test=ds['test']
gt=np.array(test['label'])
configs = [
    'zero_shot_with_heur_value_angle_cot',
    'zero_shot_with_heur_value_area_cot',
    'zero_shot_with_heur_value_angle_area_cot',    
    'few_shot_with_heur_value_angle_cot',
    'few_shot_with_heur_value_area_cot',
    'few_shot_with_heur_value_angle_area_cot'
]

In [35]:
results=[]
models=['llama3', 'mistral', '4o_mini', 'qwen_plus', '4o']

configs = [
    'zero_shot_with_heur_value_angle_cot',
    'zero_shot_with_heur_value_area_cot',
    'zero_shot_with_heur_value_angle_area_cot',    
    'few_shot_with_heur_value_angle_cot',
    #'few_shot_with_heur_value_area_cot',
    #'few_shot_with_heur_value_angle_area_cot'
]
models=['qwen_plus']
for model in models:
    print(f'Model: {model}...')
    for config in configs:
        pred=np.load(f'base/{model}/{model}_{config}.npy')
        if model=='mistral':
            pred=post_processing_old(pred)
        else:
            pred=post_processing(pred)
        metrics=metric_calculation(pred, gt)
        results.append([config, model, round(metrics[0],3), metrics[1]])
results=pd.DataFrame(results, columns=['config', 'model', 'acc', 'f1'])

Model: qwen_plus...


In [37]:
config='zero_shot_with_heur_value_angle_cot'
pred=np.load(f'base/{model}/{model}_{config}.npy')
a=post_processing(pred)
wrong_index=np.where(a==2)[0]
len(wrong_index)

321

In [49]:
pred[11]

'To determine whether the two geometries (sidewalk and road) represent the same sidewalk either fully or partially, we evaluate the condition based on **parallelism**. The key statistic provided is `min_angle`, which represents the minimum angular difference between the sidewalk and the road.\n\n### Analysis:\n- **Parallelism Condition**: If the two geometries are approximately parallel, the angular difference (`min_angle`) should be small. In this case, the provided value of `min_angle` is **0.0908 degrees**.\n  \n#### Threshold Evaluation:\nWhile no explicit threshold for `min_angle` is provided in the instruction, a typical tolerance for determining parallelism between two line geometries in geospatial analysis is often around **1–2 degrees**, depending on the application. Given that the `min_angle` value here is much smaller than even 1 degree (**0.0908 degrees**), it indicates a negligible angular difference between the sidewalk and the road.\n\nThus, based on the `min_angle` valu

In [36]:
results[results['model']==model]

,config,model,acc,f1
0,zero_shot_with_heur_value_angle_cot,qwen_plus,0.160,0.149960
1,zero_shot_with_heur_value_area_cot,qwen_plus,0.346,0.247629
2,zero_shot_with_heur_value_angle_area_cot,qwen_plus,0.313,0.306839
3,few_shot_with_heur_value_angle_cot,qwen_plus,0.684,0.471613
